In [1]:
###   Imports and Logging   ###
import sys
import logging
import math
import copy
import numpy as np
import pandas as pd
import scipy.optimize
import matplotlib.pyplot as plt
from matplotlib import __version__ as pltver
import matplotlib.ticker as ticker
from skaero.atmosphere import coesa
from skaero import __version__ as skver
import pint
units = pint.UnitRegistry()


logging.basicConfig(
    level=logging.INFO,
    format=' %(asctime)s -  %(levelname)s -  %(message)s'
)

logging.info('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
logging.info('Python version: {}'.format(sys.version))
logging.info('Author: Benjamin Crews')
logging.info('Numpy version: {}'.format(np.version.version))
logging.info('Pandas version: {}'.format(pd.__version__))
logging.info('Matplotlib version: {}'.format(pltver))
logging.info('SciKit-Aero version: {}'.format(skver))
logging.info('Pint version: {} (using standard UnitRegistry)'.format(pint.__version__))
logging.info('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')

 2020-03-31 21:06:36,362 -  INFO -  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
 2020-03-31 21:06:36,365 -  INFO -  Python version: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
 2020-03-31 21:06:36,365 -  INFO -  Author: Benjamin Crews
 2020-03-31 21:06:36,366 -  INFO -  Numpy version: 1.16.2
 2020-03-31 21:06:36,366 -  INFO -  Pandas version: 0.24.2
 2020-03-31 21:06:36,366 -  INFO -  Matplotlib version: 3.0.3
 2020-03-31 21:06:36,367 -  INFO -  SciKit-Aero version: 0.1
 2020-03-31 21:06:36,367 -  INFO -  Pint version: 0.11 (using standard UnitRegistry)
 2020-03-31 21:06:36,368 -  INFO -  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [101]:
#######################################
###   Input Rotor Characteristics   ###
#######################################
# Main Rotor
D = 35*units.ft
R = D/2           # units automatically carried over
b = 4             # [number of blades, no units]
CHORD = 10.4*units.inch
OMEGA = 43.2*units.rad/units.s
# Rotor Solidity
sol = b*CHORD.to(units.ft)/(math.pi*R)
# Rotor Area
A = math.pi*R**2
v_tip = OMEGA*R
v_tip = v_tip.to(units.ft/units.s)

GW = 5000
THRUST = 1.03*GW

######################################
###       Ambient Conditions       ###
######################################
alt = 0     # coesa does not support pint units? (untested) value in [m]
atm = coesa.table(alt) # an atmosphere at 0 height

# Give the atmosphere units and convert them to imperial.
prop_units =  [units.m, units.kelvin, units.pascal, units.kg/(units.m**3)]
h = atm[0]*prop_units[0]
h = h.to(units.ft)
T = atm[1]*prop_units[1]
T = T.to(units.fahrenheit)
p = atm[2]*prop_units[2]
p = p.to(units.psi)
rho = atm[3]*prop_units[3]
rho = rho.to(units.slug/(units.ft**3))


# Spit out the converted input to the log (and make it readable)
logging.info('')
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('Main Rotor Inputs:')
logging.info('   {name:>17}: {val:>7.3f} [{unit:~}]'.format(name='Diameter', val=D.magnitude, unit=D.units))
logging.info('   {name:>17}: {val:>7.3f} [{unit:~}]'.format(name='(constant) Chord', val=CHORD.magnitude, unit=CHORD.units))
logging.info('   {name:>17}: {val:>7.3f} [{unit}]'.format(name='Num Blades', val=b, unit=''))
logging.info('   {name:>17}: {val:>7.3f} [{unit:~}]'.format(name='Rotational Speed', val=OMEGA.magnitude, unit=OMEGA.units))
logging.info('   {name:>17}: {val:>7.3f} [{unit:~}]'.format(name='Tip Speed', val=v_tip.magnitude, unit=v_tip.units))
logging.info('   {name:>17}: {val:>7.3f} [{unit:~}]'.format(name='Area', val=A.magnitude, unit=A.units))
logging.info('   {name:>17}: {val:>7.3f} [{unit:~}]'.format(name='Solidity', val=sol.magnitude, unit=sol.units))
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('Atmospheric Properties:')
logging.info('   {name:>17}: {val:>9.5f} [{unit:~}]'.format(name='Altitude', val=h.magnitude, unit=h.units))
logging.info('   {name:>17}: {val:>9.5f} [{unit:~}]'.format(name='Temperature', val=T.magnitude, unit=T.units))
logging.info('   {name:>17}: {val:>9.5f} [{unit:~}]'.format(name='Pressure', val=p.magnitude, unit=p.units))
logging.info('   {name:>17}: {val:>9.5f} [{unit:~}]'.format(name='Density', val=rho.magnitude, unit=rho.units))
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('')

 2020-04-01 19:54:27,660 -  INFO -  
 2020-04-01 19:54:27,661 -  INFO -  -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
 2020-04-01 19:54:27,662 -  INFO -  Main Rotor Inputs:
 2020-04-01 19:54:27,664 -  INFO -              Diameter:  35.000 [ft]
 2020-04-01 19:54:27,664 -  INFO -      (constant) Chord:  10.400 [in]
 2020-04-01 19:54:27,665 -  INFO -            Num Blades:   4.000 []
 2020-04-01 19:54:27,666 -  INFO -      Rotational Speed:  43.200 [rad / s]
 2020-04-01 19:54:27,666 -  INFO -             Tip Speed: 756.000 [ft / s]
 2020-04-01 19:54:27,667 -  INFO -                  Area: 962.113 [ft ** 2]
 2020-04-01 19:54:27,667 -  INFO -              Solidity:   0.063 []
 2020-04-01 19:54:27,667 -  INFO -  -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
 2020-04-01 19:54:27,668 -  INFO -  Atmospheric Properties:
 2020-04-01 19:54:27,668 -  INFO -              Altitude:   0.00000 [ft]
 2020-04-01 19:54:27,669 -  INFO -           Temperature:  59.00000 [°F]
 2020-04-01 19:54:27,669 -  INF

In [105]:
######################################
###  Blade Downwash Distribution   ###
###           (HOGE)               ###
######################################
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('Beginning HOGE Downwash calculations...')
# To iterate on downwash solution, a function to calculate rotor thrust must
# be defined to take tip_angle as an input.
# Then, scipy.minimize can be used to reach zero error.

# TEMP: Inputs
delta_1 = -0.0216
delta_2 = 0.4
k_i = 1.1
CLIMB_RATE = 0

def thrust_generator(num: int,
                    TWIST: int,
                    theta_tip: float,
                    sol: float,
                    LIFT_CURVE_SLOPE: float,
                    v_tip: float,
                    CLIMB_RATE: float,
                    R: float,
                    B: float = 1.0
                   ) -> pd.DataFrame:
    '''
    Note: Despite the name, this function is not a pythonic generator.
    
    This function generates a thrust table for a rotor,
    using Blade-Element-Momentum Theory.
    
    This model assumes a constant-chord blade with no root cut-out,
    and a linear twist.
    '''
    df = pd.DataFrame(np.linspace(0.001, B, num), columns=['x'])
    df['theta'] = TWIST*(df.x - 1) + theta_tip

    c_1 = sol*LIFT_CURVE_SLOPE*v_tip/16
    c_2 = CLIMB_RATE/2 + sol*LIFT_CURVE_SLOPE*v_tip/16
    c_3 = 4*CLIMB_RATE**2/(sol*LIFT_CURVE_SLOPE*v_tip)

    # create a temporary variable which is a function of
    # blade stationand blade-element angle
    # used to calculate the final induced velocity.
    df['v_step'] = 2*(np.radians(df.theta)*df.x*v_tip - CLIMB_RATE)
    df['v_induced'] = c_2 * (-1 + np.sqrt(1 + df.v_step/(c_3 + CLIMB_RATE + c_1)))
    # Calculate the differential thrust of each blade element
    # Until I know a way to elegantly do this with pandas
    # convert everything to np arrays and loop through
    # TODO: cleanup code
    x_temp = np.array(df.x)
    x_temp = np.append(x_temp, 0)
    vi_temp = np.array(df.v_induced)
    vi_temp[-1] = 0
    df['v_induced'] = -vi_temp
    dT_temp = np.array([])
    for i, vi in enumerate(vi_temp):
        dT_temp = np.append(dT_temp, 2*math.pi*rho*(CLIMB_RATE+vi)*vi*x_temp[i]*R*R*(x_temp[i+1] - x_temp[i]) )


    df['dT'] = dT_temp
    
    return df


def T_err(theta, req, args):
    '''
    Calculates the thrust error for iteration.
    '''
    df = thrust_generator(num = args[0],
                          TWIST = args[1],
                          theta_tip = theta,
                          sol = args[2],
                          LIFT_CURVE_SLOPE= args[3],
                          v_tip = args[4],
                          CLIMB_RATE = args[5],
                          R = args[6],
                          B = args[7]
                         )
    
    calcd = df.dT.sum()
    err = abs(req-calcd)/req
    return err
    

###   Airfoil Lift factor correction from 2d to 3d   ###

# TODO: a_0 = 2*pi ; a = a_0 / (1 + 57.3*a_0/(pi*AR))
# 2pi is the ideal, infinite airfoil with no edge effects.
# This equation transforms it into a 3d lift-curve-slope
# 57.3 means a is in cl/rad, but 2pi is cl/deg. So convert to rad first.

# TODO: AR = R/C
# Aspect ratio is Rotor radius over the equivalent chord.
ASPECT_RATIO = R/CHORD

a_0 = 2*math.pi*units.deg
a0 = a_0.to(units.rad)
a = a_0 / (1 + 57.3*a_0/(math.pi*ASPECT_RATIO))



###   Compressibility correction factor   ###

# TODO: 1st term of 3-term profile drag is for compressible flow only.
#       need to correct this to account for compressibilty of such a fast
#       moving rotor.
#       delta_0, the 1st drag term, is: d_0i/(sqrt(abs(Mach#**2 - 1)))
#       d_0i, the uncorrected 1st term, is given.

# The Mach num here is the mach @ 80% blade station = v_tip*0.8/c_sound
# c_sound is the local speed of sound, aka sqrt(gamma*R*T) = sqrt(1.4*1716.4*T), where T is in Rankine.
c_sound = math.sqrt(1.4*1716.4*T.to(units.rankine).magnitude)
mach_08 = v_tip.magnitude*0.8/c_sound
delta_0 = 0.0080/math.sqrt(abs(mach_08**2 - 1))


# TODO: Get Ct from Momentum theory = T/(rho*A*v_tip**2)
Ct_mom = THRUST/(rho*A*v_tip**2)
Ct_mom = Ct_mom.magnitude

# TODO: Get B correction for tip-loss = 1 - (sqrt(2*Ct)/b)
B = 1 - math.sqrt(2*Ct_mom)/b

# TODO: ideal to linear twist correction
#       Power*correction = linear-twist power, based on ideal momentum theory calculations.
#       HP_i*k_i = HP_lin , k_i is given, usually 5-10% more (k_i=1.1)
#       HP_i = Power (converted to horsepower)
#       Power = C_p*rho*A*v_tip**3
#       C_p = C_q = 3-term drag model


C_qi = 0.5*Ct_mom*math.sqrt( (CLIMB_RATE/(OMEGA.magnitude*R.magnitude))**2 + 2*Ct_mom/B**2 )
C_qvi = CLIMB_RATE*Ct_mom/(2*OMEGA.magnitude*R.magnitude)
C_q0 = sol.magnitude*delta_0/8
C_q1 = (2*delta_1/(3*a.magnitude))*(Ct_mom/B**2)
C_q2 = (4*delta_2/(sol.magnitude*a.magnitude**2))*(Ct_mom/B**2)**2

C_q = C_qi + C_w + C_q0 + C_q1 + C_q2

P = C_q*rho.magnitude*A.magnitude*v_tip.magnitude**3
HP_i = P/550   # Power, in horsepower
HP_i = HP_i*units.horsepower
HP = HP_i * k_i
Q = HP.magnitude*550*R.magnitude/v_tip.magnitude
Q = Q*units.lb*units.ft


# To do calculations, we pulled only the magnitudes
# add the units back in, for nice printing. (HACK, please correct later. No time now. TODO)
a = a/units.deg
P = P*units.ft*units.lb/units.s



logging.info('   {name:>18}: {val:>9.5f} [{unit:~}]'.format(name='3D Lift Coef.', val=a.magnitude, unit=a.units))
logging.info('')
logging.info('Compressibility Correction: ')
logging.info('   {name:>18}: {val:>9.5f} [{unit:}]'.format(name='0.8STA Mach Num', val=mach_08, unit=''))
logging.info('   {name:>18}: {val:>9.5f} [{unit:}]'.format(name='delta_0', val=delta_0, unit=''))
logging.info('')
logging.info('According to pure Momentum Theory: ')
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='Coef. of Thrust', val=Ct_mom, unit=''))
logging.info('   {name:>18}: {val:>10.3f} [{unit:}]'.format(name='Tip-Loss Factor', val=B, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_qi', val=C_qi, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_qvi', val=C_w, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_q0', val=C_q0, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_q1', val=C_q1, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_q2', val=C_q2, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='Coef. of Pwr/Torq', val=C_q, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:~}]'.format(name='Total, Ideal Power', val=P.magnitude, unit=P.units))
logging.info('   {name:>18}: {val:>10.3f} [{unit:~}]'.format(name='Corrected Power', val=HP.magnitude, unit=HP.units))
logging.info('   {name:>18}: {val:>10.3f} [{unit:~}]'.format(name='Total Torque', val=Q.magnitude, unit=Q.units))
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('')

## Note: With all the correction factors above, we have a roughly approximate solution
#        for realistic power, using just momentum theory. The blade-element portion
#        is not necessary yet.

    
args = (50, -12, sol.magnitude, 2*math.pi, v_tip.magnitude, CLIMB_RATE, R.magnitude, 1)
res = scipy.optimize.minimize(T_err, x0=2, args=(THRUST, args), tol=0.01, options={'disp':True})
logging.info('  SciPy Optimization resulted in theta_tip = {:.3f}, after {} iterations.'.format(res.x[0], res.nfev))
logging.info('  Using optimization results to calculate Main Rotor data...')
logging.info('')
theta = res.x[0]
MR = thrust_generator(num = args[0],
                          TWIST = args[1],
                          theta_tip = theta,
                          sol = args[2],
                          LIFT_CURVE_SLOPE= args[3],
                          v_tip = args[4],
                          CLIMB_RATE = args[5],
                          R = args[6],
                          B = args[7]
                         )

logging.info(f'   Total Main Rotor Thrust = {MR.dT.sum():10.2f} [lb]')
logging.info('')

###   Main Rotor Torque Calculations   ###
# TODO:

logging.info('Blade Station Downwash Table: ')
print(MR)

 2020-04-01 19:56:05,645 -  INFO -  -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
 2020-04-01 19:56:05,645 -  INFO -  Beginning HOGE Downwash calculations...
 2020-04-01 19:56:05,647 -  INFO -          3D Lift Coef.:   5.71690 []
 2020-04-01 19:56:05,648 -  INFO -  
 2020-04-01 19:56:05,649 -  INFO -  Compressibility Correction: 
 2020-04-01 19:56:05,649 -  INFO -        0.8STA Mach Num:   0.54174 []
 2020-04-01 19:56:05,649 -  INFO -                delta_0:   0.00952 []
 2020-04-01 19:56:05,650 -  INFO -  
 2020-04-01 19:56:05,650 -  INFO -  According to pure Momentum Theory: 
 2020-04-01 19:56:05,652 -  INFO -        Coef. of Thrust:  3.940E-03 []
 2020-04-01 19:56:05,653 -  INFO -        Tip-Loss Factor:      0.978 []
 2020-04-01 19:56:05,653 -  INFO -                   C_qi:  1.789E-04 []
 2020-04-01 19:56:05,654 -  INFO -                  C_qvi:  0.000E+00 []
 2020-04-01 19:56:05,655 -  INFO -                   C_q0:  7.502E-05 []
 2020-04-01 19:56:05,655 -  INFO -                

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 78
         Gradient evaluations: 26
           x      theta      v_step  v_induced          dT
0   0.001000  21.383529    0.564298  -0.280054    0.000007
1   0.021388  21.138876   11.930984  -5.233839    0.054631
2   0.041776  20.894223   23.034412  -9.237872    0.332430
3   0.062163  20.649570   33.874585 -12.657868    0.928730
4   0.082551  20.404917   44.451501 -15.668523    1.889788
5   0.102939  20.160263   54.765160 -18.369651    3.239033
6   0.123327  19.915610   64.815562 -20.824731    4.987121
7   0.143714  19.670957   74.602709 -23.077142    7.136719
8   0.164102  19.426304   84.126598 -25.158107    9.685109
9   0.184490  19.181651   93.387231 -27.090996   12.625742
10  0.204878  18.936998  102.384607 -28.893845   15.949228
11  0.225265  18.692345  111.118727 -30.580923   19.644007
12  0.245653  18.447692  119.589590 -32.163745   23.696817
13 

In [106]:
#######################################
###   Forward Flight Calculations   ###
#######################################
logging.info('Beginning Forward Flight Calculations...')

# Max Airspeed, in knots
VNE = 160
fe = 12.4 * units.ft**2     # Equivalent Flat-Plate Drag Area

df = pd.DataFrame(data=np.linspace(20, VNE, num=20), columns=['Airspeed'])
df['q'] = 0.5*rho.magnitude*(df.Airspeed * 1.689)**2    # 1.689 is simply the conversion factor from kts to ft/s
df['mu'] = df.Airspeed * 1.689 / v_tip
v_0 = math.sqrt(THRUST/(2*rho.magnitude*math.pi*R.magnitude**2))
df['v_if'] = v_0 * (-0.5*(df.Airspeed/v_0)**2 + np.sqrt((df.Airspeed/v_0)**4 / 4 + 1))
df['Ct'] = THRUST/(rho.magnitude*A.magnitude*v_tip.magnitude**2)
df['tc'] = 2*df.Ct/sol.magnitude
df['tc_lower'] = -0.6885*df.Ct/sol.magnitude + 0.3555
df['F'] = ((df.Ct/sol.magnitude)/(1-df.mu)**2) * (1 + fe*df.q/GW) - 0.1376
df['del_cds'] = 18.3*(1-df.mu)**2*df.F**3
df['MDD'] = 0.82 - 2.4*df.Ct/sol.magnitude

df

 2020-04-01 19:56:13,855 -  INFO -  Beginning Forward Flight Calculations...
D:\Czarified\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:876: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  v = np.array(v, copy=False)
D:\Czarified\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1243: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  subarr = np.array(values, dtype=dtype, copy=copy)


,Airspeed,q,mu,v_if,Ct,tc,tc_lower,F,del_cds,MDD
0,20.000000,1.356121,0.044683,28.121098,0.00394,0.124978,0.312476,-0.068898,-5.462274e-03,0.670026
1,27.368421,2.539440,0.061145,24.202585,0.00394,0.124978,0.312476,-0.066260,-4.692477e-03,0.670026
2,34.736842,4.090903,0.077607,20.089759,0.00394,0.124978,0.312476,-0.063408,-3.969359e-03,0.670026
3,42.105263,6.010510,0.094069,16.290032,0.00394,0.124978,0.312476,-0.060325,-3.297101e-03,0.670026
4,49.473684,8.298260,0.110530,13.088497,0.00394,0.124978,0.312476,-0.056990,-2.679838e-03,0.670026
5,56.842105,10.954154,0.126992,10.540418,0.00394,0.124978,0.312476,-0.053381,-2.121543e-03,0.670026
6,64.210526,13.978192,0.143454,8.566996,0.00394,0.124978,0.312476,-0.049474,-1.625871e-03,0.670026
7,71.578947,17.370373,0.159916,7.049219,0.00394,0.124978,0.312476,-0.045242,-1.195943e-03,0.670026
8,78.947368,21.130698,0.176378,5.876388,0.00394,0.124978,0.312476,-0.040654,-8.340757e-04,0.670026
9,86.315789,25.259167,0.192840,4.960623,0.00394,0.124978,0.312476,-0.035677,-5.414147e-04,0.670026
